# Simulate Tensor

In [1]:
# from IPython.core.alias import Instance
import numpy as np

def traverse(o, tree_types=(list, tuple, np.ndarray), index = None, nest_layer = 100):
    if isinstance(o, tree_types) and nest_layer > 0:
        for idx, value in enumerate(o):
            new_index = index + [idx] if type(index) == list else [idx]
            for subvalue in traverse(value, tree_types, new_index, nest_layer - 1):
                yield subvalue
    else:
        if not isinstance(o, tree_types): 
            length = None
        else:
            length = len(o)
        yield index, length, o

In [2]:
# TODO: based on fld_name, get input_tensor
import numpy as np
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

###############
# 'B-P-EC:Diag-Rec:V-VdftGrn'
# B: batch
# P: patient
# EC:Diag: Encounter's DiagInfo
# Rec:V: a Rec's Value: E11
# VdftGrn: Value to a list of Grn. ICD-10 [E, 1, 1]
fld_name = 'B-P-EC-DiagRec:DiagV-DiagVdftGrn'
###############


# fld_name = 'B2St2TkGrn'
layers_num = len(fld_name.split('-'))
print(layers_num)

# max_list = df['max'].astype(int).to_list()
prefix = [np.array(B_lenP).max(), max(B2P_lnEC)] 
max_list = [np.array(B_lenP).max(), max(B2P_lnEC)] + list(np.random.randint(1, 10, layers_num - len(prefix) - 1)) + [vocab_size]
print(max_list)

5
[3, 6, 2, 2, 100]


In [3]:
def get_next_info(layer, current_info, max_list):
    current_max = max_list[layer]
    next_max = max_list[layer + 1]
    # to do: get next_info
    next_info = np.zeros(list(np.array(current_info).shape) + [current_max]).astype(int)
    # print(next_info.shape)
    for element in list(traverse(current_info, nest_layer = layer)):
        idx, leng, value = element
        # print(next_info[tuple(idx)])
        next_info[tuple(idx)][:value] = np.random.randint(1, next_max + 1, value)

    # print(next_info)
    return next_info

In [4]:
def get_simulated_tensor_from_fldname(fld_name, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size):
    layers_num = len(fld_name.split('-'))
    # print(layers_num)

    # max_list = df['max'].astype(int).to_list()
    prefix = [np.array(B_lenP).max(), max(B2P_lnEC)] 
    max_list = [np.array(B_lenP).max(), max(B2P_lnEC)] + list(np.random.randint(1, 10, layers_num - len(prefix) - 1)) + [vocab_size]
    # print(max_list)

    init_info = np.array(B2P_lnEC)
    
    for layer_idx in range(prefix_layers_num - 1, layers_num - 1):
        print(layer_idx)
        current_info = init_info
        next_info = get_next_info(layer_idx, current_info, max_list)
        # print(next_info)
        init_info = next_info

        print(layer_idx, '-->', current_info.shape)
        print(layer_idx + 1, '-->', next_info.shape)


    fld_tensor_idx = next_info
    # print(fld_tensor.shape)
    return fld_tensor_idx


In [5]:
B_lenP = 3
B2P_lnEC = [6, 5, 2] # 
prefix_layers_num = 2
vocab_size = 100

fld_name = 'B-P-EC-Diag:DiagV-DiagVdftGrn'
fld_tensor_idx = get_simulated_tensor_from_fldname(fld_name, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(fld_tensor_idx.shape)
# fld_tensor_idx

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 6)
3
3 --> (3, 6, 6)
4 --> (3, 6, 6, 4)
(3, 6, 6, 4)


In [6]:
fld_tensor_idx.shape

# ( 3,6,      4,    3)
# 'B-P-EC:Diag-DiagRec:V-VdftGrn' [E, 1, 1]
# fld_tensor_idx

(3, 6, 6, 4)

# info_idx, info, leng_mask, leng

1. Current Layer:
```python
leng_mask = info_idx == 0 # or info_idx != 0
leng = leng_mask.sum(-1)
```

2. Transfer

```python
old_leng = leng
```


3. Next Layer
```python
leng_mask = old_leng != 0
leng = (leng_mask == 0).sum(-1)
```

In [7]:
########################
B_lenP = 3
B2P_lnEC = [6, 4, 3] # 
prefix_layers_num = 2
vocab_size = 100
########################

fullname = 'B-P-EC-Diag:DiagV-DiagVdftGrn'
# fld_name = 'B2St2TkGrn'
data = get_simulated_tensor_from_fldname(fullname, B_lenP, B2P_lnEC, prefix_layers_num, vocab_size)
print(data.shape)
# fld_tensor_idx

1
1 --> (3,)
2 --> (3, 6)
2
2 --> (3, 6)
3 --> (3, 6, 8)
3
3 --> (3, 6, 8)
4 --> (3, 6, 8, 9)
(3, 6, 8, 9)


## 'B-P-EC-Diag:DiagV-DiagVdftGrn'

In [8]:
data

array([[[[ 51,  73,  63, ...,   0,   0,   0],
         [ 89,  15,   0, ...,   0,   0,   0],
         [  2,  16,  44, ...,  15,   0,   0],
         ...,
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],

        [[ 86,   3,  29, ...,   0,   0,   0],
         [ 54,  49,  49, ...,   0,   0,   0],
         [ 59,  36,  68, ...,   0,   0,   0],
         ...,
         [ 38,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0],
         [  0,   0,   0, ...,   0,   0,   0]],

        [[ 78,  41,   0, ...,   0,   0,   0],
         [100,  53,   5, ...,   0,   0,   0],
         [ 67,  54,  98, ...,   0,   0,   0],
         ...,
         [ 87,  12,   0, ...,   0,   0,   0],
         [ 17,  65,  99, ...,  62,  10,  51],
         [ 97,  39,  23, ...,   0,   0,   0]],

        [[ 62,   0,   0, ...,   0,   0,   0],
         [ 18,  68,  98, ...,   0,   0,   0],
         [  0,   0,   0, ...,   

In [9]:
import torch
info_idx = torch.LongTensor(data)
print(info_idx.shape)

holder = info_idx

leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)
pos_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
print(pos_idx.shape)

leng_mask


torch.Size([3, 6, 8, 9])
torch.Size([3, 6, 8, 9])
torch.Size([3, 6, 8, 9])


tensor([[[[False, False, False,  ...,  True,  True,  True],
          [False, False,  True,  ...,  True,  True,  True],
          [False, False, False,  ..., False,  True,  True],
          ...,
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True]],

         [[False, False, False,  ...,  True,  True,  True],
          [False, False, False,  ...,  True,  True,  True],
          [False, False, False,  ...,  True,  True,  True],
          ...,
          [False,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True]],

         [[False, False,  True,  ...,  True,  True,  True],
          [False, False, False,  ...,  True,  True,  True],
          [False, False, False,  ...,  True,  True,  True],
          ...,
          [False, False,  True,  ...,  True,  True,

## 'B-P-EC-Diag:DiagV'

or 'B-P-EC-Diag:DiagV:DiagVdftGrn'

or 'B-P-EC-Diag:DiagV'

In [10]:
# next layer
holder = leng

leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)

leng_mask

torch.Size([3, 6, 8])


tensor([[[False, False, False, False,  True,  True,  True,  True],
         [False, False, False, False, False, False,  True,  True],
         [False, False, False, False, False, False, False, False],
         [False, False,  True,  True,  True,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False, False,  True,  True,  True,  True]],

        [[False, False, False, False,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False,

## 'B-P-EC'

or 'B-P-EC:Diag:DiagV:DiagVdftGrn'

or 'B-P-EC:Diag:DiagV'

or 'B-P-EC:Diag'

or 'B-P-EC'


In [11]:
holder = leng
leng_mask = holder == 0
print(leng_mask.shape)
leng = (leng_mask == 0).sum(-1)

leng_mask

torch.Size([3, 6])


tensor([[False, False, False, False, False, False],
        [False, False, False, False,  True,  True],
        [False, False, False,  True,  True,  True]])

## B-P

In [12]:
holder = leng
leng_mask = holder == 0

print(leng_mask.shape)
leng_mask

torch.Size([3])


tensor([False, False, False])

In [13]:
leng = (leng_mask == 0).sum(-1)
leng

tensor(3)

In [14]:
B2P_lnEC

[6, 4, 3]